In [1]:
import numpy as np
import torch
import torch.nn as nn

from lamp import *

In [64]:
datax = [
    [[0.1517, 0.5443, 0.1664, 0.2968, 0.1047, 0.2303, 0.7809, 0.8742]],
    [[0.6482, 0.8415, 0.3437, 0.5621, 0.8318, 0.5978, 0.2179, 0.5826]],
    # [[0.0959, 0.5683, 0.8281, 0.2220, 0.7972, 0.3602, 0.9269, 0.3220]],
    # [[0.2017, 0.4027, 0.9019, 0.3093, 0.2855, 0.0226, 0.3115, 0.5141]],
]

# data shape
batch, chan_in, length = np.array(datax).shape

# conv shape
k_size = 3
chan_out = 2

params = np.random.rand(k_size, chan_in, chan_out)
np_input = np.array(datax).reshape(batch, length, chan_in)
th_input = torch.Tensor(datax)

In [4]:
conv = Conv1D(k_size, chan_in, chan_out)
output = conv.forward(np_input)
print(output.shape)
np.round_(output.reshape(batch, chan_out, -1), decimals=4)

(4, 6, 2)


array([[[ 0.0542,  0.022 , -0.006 ,  0.0323,  0.0232,  0.0138],
        [ 0.0046,  0.024 ,  0.0869,  0.0743,  0.1372,  0.0896]],

       [[ 0.0538,  0.0445,  0.0089,  0.0603,  0.1016,  0.0841],
        [ 0.085 ,  0.067 ,  0.0014,  0.0316,  0.0212,  0.0593]],

       [[ 0.1247,  0.0822,  0.0483,  0.0328,  0.0208,  0.0804],
        [ 0.0905,  0.0428,  0.0493,  0.0935,  0.0866,  0.0416]],

       [[ 0.1061,  0.0878,  0.079 ,  0.0404, -0.0264,  0.0351],
        [ 0.001 ,  0.0071,  0.0055,  0.0305,  0.0769,  0.0504]]])

In [5]:
conv_ = nn.Conv1d(chan_in, chan_out, k_size)
output = conv_(th_input)
output.shape

torch.Size([4, 2, 6])

In [11]:
maxp = MaxPool1D(k_size, stride=1)
# output = maxp.forward(np_input)
# output.reshape(batch, chan_in, -1)

In [7]:
maxp_ = nn.MaxPool1d(k_size, stride=1)
output = maxp_(th_input)
output

tensor([[[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742]],

        [[0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978]],

        [[0.8281, 0.8281, 0.8281, 0.7972, 0.9269, 0.9269]],

        [[0.9019, 0.9019, 0.9019, 0.3093, 0.3115, 0.5141]]])

In [54]:
stride = 3
d_out = (length - k_size) // stride + 1
res = np.zeros((batch, d_out, chan_in))

for k in range(d_out):
    t1, t2 = stride * k, 2 * (k_size // 2) + k * stride + 1
    res[:, k, :] = np.amax(np_input[:, t1:t2, :], axis=1)

res.reshape(batch, chan_in, -1)

array([[[0.5443, 0.2968]]])

In [69]:
res = np.zeros((batch, d_out, chan_in))
print(np_input)

for k in range(d_out):
    t1, t2 = stride * k, 2 * (k_size // 2) + k * stride + 1
    amax = np.amax(np_input[:, t1:t2, :], axis=1).flatten()
    # print(amax)
    # print(np.where(np_input == amax, 1, 0))
    idx = np.where(np.all(np_input == amax, axis=-1))[0]
    print(idx)
    # res[:, k, :] = np_input[:, idx:]

# res.reshape(batch, chan_in, -1)

[[[0.1517]
  [0.5443]
  [0.1664]
  [0.2968]
  [0.1047]
  [0.2303]
  [0.7809]
  [0.8742]]

 [[0.6482]
  [0.8415]
  [0.3437]
  [0.5621]
  [0.8318]
  [0.5978]
  [0.2179]
  [0.5826]]]
[]
[]


In [8]:
input = maxp_(th_input)

m6 = nn.Flatten()
output = m6(input)
output.size(), output

(torch.Size([4, 6]),
 tensor([[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742],
         [0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978],
         [0.8281, 0.8281, 0.8281, 0.7972, 0.9269, 0.9269],
         [0.9019, 0.9019, 0.9019, 0.3093, 0.3115, 0.5141]]))

In [9]:
input = maxp.forward(np_input)

m5 = Flatten()
output = m5.forward(input)
output.shape, output

NameError: name 'maxp' is not defined

In [ ]:
np.arange

In [152]:
stride = 1
m1 = Conv1D(k_size, chan_in, chan_out, stride=stride)
delta = m1.forward(np_input)
print(delta.shape)
d_out = length

res = np.zeros((batch, length, chan_in))

delta


for kd, kr in enumerate(list(range(0, d_out, stride))):
    print("kd", kd, " kr", kr)
    t1, t2 = 0, k_size
    d1 = -kd - 2 + k_size
    d2 = -kd - 2

    if kd == 0:
        t1, t2 = 0, kd + 1
        d1, d2 = None, -1

    elif kd < k_size - 1:
        print("1")
        t1, t2 = 0, kd + 1
        d1, d2 = -1, -kd - 2

    elif kd > d_out - (k_size + 1):
        print("2")
        t1, t2 = k_size - d_out + kd, k_size
        d1, d2 = -kd + 2, 0

    print(d1, d2, kr, kd)
    print(np.flip(params, axis=1)[np.newaxis, t1:t2, :, :])
    print(delta[:, d2:d1, np.newaxis, :])

    res[:, kr, :] = np.sum(
        np.flip(params, axis=1)[np.newaxis, t1:t2, :, :]
        * delta[:, d2:d1, np.newaxis, :],
        axis=(1, 3),
    )

res.shape, np_input.shape


(2, 6, 2)
kd 0  kr 0
None -1 0 0
[[[[0.87594111 0.54180094]]]]
[[[[-0.05562612 -0.07655721]]]


 [[[-0.04782823 -0.03788913]]]]
kd 1  kr 1
1
-1 -3 1 1
[[[[0.87594111 0.54180094]]

  [[0.50938715 0.6180765 ]]]]
[[[[-0.02129233 -0.01353966]]

  [[-0.03897679 -0.06823244]]]


 [[[-0.05667553 -0.04375157]]

  [[-0.04696213 -0.00164538]]]]
kd 2  kr 2
-1 -4 2 2
[[[[0.87594111 0.54180094]]

  [[0.50938715 0.6180765 ]]

  [[0.41886727 0.15917641]]]]
[[[[-0.01461547 -0.00681058]]

  [[-0.02129233 -0.01353966]]

  [[-0.03897679 -0.06823244]]]


 [[[-0.05446176 -0.06859957]]

  [[-0.05667553 -0.04375157]]

  [[-0.04696213 -0.00164538]]]]
kd 3  kr 3
-2 -5 3 3
[[[[0.87594111 0.54180094]]

  [[0.50938715 0.6180765 ]]

  [[0.41886727 0.15917641]]]]
[[[[-0.0335419  -0.01349704]]

  [[-0.01461547 -0.00681058]]

  [[-0.02129233 -0.01353966]]]


 [[[-0.05739264 -0.03056941]]

  [[-0.05446176 -0.06859957]]

  [[-0.05667553 -0.04375157]]]]
kd 4  kr 4
-3 -6 4 4
[[[[0.87594111 0.54180094]]

  [[0.50938715 0.

((2, 8, 1), (2, 8, 1))